In [1]:

## kosis 현재 인구 데이터 가공

import pandas as pd
import _config
import _codes
import _util

def popExcelToCsv(file_nm, year):
    sido_cd_map = {info['sido_nm']:info['sido_cd'] for info in _codes.sido_codes}
    sido_sgg_cd_map = {}

    #해당년도 해당 변수 불러오기
    if year < 2005: sido_sgg_cd_map = _util.process_sgg_codes(_codes.sgg_codes_2000, 1)
    elif year < 2010: sido_sgg_cd_map = _util.process_sgg_codes(_codes.sgg_codes_2005, 1)
    elif year < 2015: sido_sgg_cd_map = _util.process_sgg_codes(_codes.sgg_codes_2010, 1)
    elif year < 2020: sido_sgg_cd_map = _util.process_sgg_codes(_codes.sgg_codes_2015, 1)
    elif year < 2023: sido_sgg_cd_map = _util.process_sgg_codes(_codes.sgg_codes_2020, 1)
    elif year >= 2023: sido_sgg_cd_map = _util.process_sgg_codes(_codes.sgg_codes_2023, 1)

    else: raise Exception

    df = pd.read_csv(_config.pop_read_path + f'\\{file_nm}.csv', encoding=_config.euc_kr)
    keyword = '년'

    sido_cd = None
    #sido_cd row마다 부여
    for i in range(len(df)):
        df.loc[i, '년도'] = year
        sgg = df.loc[i, '행정구역(시군구)별'].strip()

        # 시도 코드 매핑
        sido_cd = sido_cd_map.get(sgg, sido_cd)
        df.loc[i, 'sido_cd'] = f'`{str(sido_cd)}'  # 시도 코드 할당

        # 시군구 코드 매핑2
        sido = sido_sgg_cd_map.get(sido_cd, {})
        sido = dict(sido)
        sgg_cd = sido.get(sgg, None)

        if sgg == '전라북도': sgg_cd = '35'
        df.loc[i, 'sgg_cd'] = f'`{str(sgg_cd)}'
            
    # year 부여
    df['year'] = f'`{year}'

    _cols = {}
    for col in df.columns:
        if keyword in col and f'{keyword}도' not in col:
            _cols[col] = 'cnt'
            continue
        if col.count('시군구') > 0: 
            _cols[col] = 'sgg_nm'
            continue
        if col.count('성별') > 0: 
            _cols[col] = 'gender'
            continue
        if '6' not in col: _cols[col] = col

    df.rename(columns=_cols, inplace=True)
    
    # 로우를 컬럼으로 변경
    df = df.pivot_table(
        index=['sido_cd', 'sgg_cd', 'sgg_nm', 'gender', 'year'],
        columns='연령별',
        values='cnt',
        aggfunc='sum',
        fill_value=0
    )
    df.reset_index(inplace=True)

    # 열 이름 목록 생성
    _list = []
    for col in df.columns:
        if '85세 이상' in col or '80세 이상' in col:
            continue
        _list.append(col)
    
    df = df[_list]

    # 나이를 기준으로 정렬
    non_age_columns = ['sido_cd', 'sgg_cd', 'sgg_nm', 'gender', 'year']
    age_columns = [col for col in df.columns if col not in non_age_columns]
    age_tuples = [(col, _util.extract_number(col)) for col in age_columns] #extract age
    age_tuples_sorted = sorted(age_tuples, key=lambda x: x[1]) #sort
    sorted_columns = non_age_columns + [age[0] for age in age_tuples_sorted]
    df = df[sorted_columns]

    try:
        df.to_csv(_config.pop_write_path + f'\\{file_nm}.csv', encoding=_config.euc_kr, index=False)
    except:
        print(f'해당 파일쓰기 실행 중 에러 발생 {file_nm}')
        return False
    
    print(f'{file_nm}.csv 파일 생성완료')

    return True

suc_cnt = 0
fail_cnt = 0
years = list(range(2000, 2024))
for year in years:
    year_str = str(year)[2:4]
    if popExcelToCsv(year_str + '년도 인구', year): suc_cnt += 1
    else: fail_cnt += 1

print('성공', suc_cnt, '실패', fail_cnt)


00년도 인구.csv 파일 생성완료
01년도 인구.csv 파일 생성완료
02년도 인구.csv 파일 생성완료
03년도 인구.csv 파일 생성완료
04년도 인구.csv 파일 생성완료
05년도 인구.csv 파일 생성완료
06년도 인구.csv 파일 생성완료
07년도 인구.csv 파일 생성완료
08년도 인구.csv 파일 생성완료
09년도 인구.csv 파일 생성완료
10년도 인구.csv 파일 생성완료
11년도 인구.csv 파일 생성완료
12년도 인구.csv 파일 생성완료
13년도 인구.csv 파일 생성완료
14년도 인구.csv 파일 생성완료
15년도 인구.csv 파일 생성완료
16년도 인구.csv 파일 생성완료
17년도 인구.csv 파일 생성완료
18년도 인구.csv 파일 생성완료
19년도 인구.csv 파일 생성완료
20년도 인구.csv 파일 생성완료
21년도 인구.csv 파일 생성완료
22년도 인구.csv 파일 생성완료
23년도 인구.csv 파일 생성완료
성공 24 실패 0


In [60]:
# kosis 추세 인구 데이터 가공

import _config
import _util
import _codes
import pandas as pd

df = pd.read_csv(_config.pop_read_path + '\\' + '성_및_연령별_추계인구_1세별__5세별____전국_20240524100134' + '.csv', encoding=_config.euc_kr)

number_cols = [col for col in df.columns if str(col).isnumeric()]
not_number_cols = [col for col in df.columns if not str(col).isnumeric()]

df = pd.melt(
    df
    , id_vars=not_number_cols
    , value_vars=number_cols
    , var_name='year'
    , value_name='cnt'
)

_list = []
for col in df.columns:
    if col != '가정별': _list.append(col)

df = df[_list]
df = pd.DataFrame(df)

_cols = {}
for col in df.columns:
    if col.count('성별'): _cols[col] = 'gender'
df.rename(columns=_cols, inplace=True)

df['sido_cd'] = '`00'
df['sgg_cd'] = '`None'
df['sgg_nm'] = '전국'
df['year'] = '`' + df['year'].astype(str)  # 이스케이프 문제 해결

df = df.pivot_table(
    index=['sido_cd', 'sgg_cd', 'sgg_nm', 'gender', 'year']
    , columns='연령별'
    , values='cnt'
    , aggfunc='sum'
    , fill_value=0
    # , dropna=False
)
df.reset_index(inplace=True)

#age sort
non_age_columns = ['sido_cd', 'sgg_cd', 'sgg_nm', 'gender', 'year']
age_columns = [col for col in df.columns if col not in non_age_columns]
age_tuples = [(col, _util.extract_number(col)) for col in age_columns] #extract age
age_tuples_sorted = sorted(age_tuples, key=lambda x: x[1]) #sort
sorted_columns = non_age_columns + [age[0] for age in age_tuples_sorted]
df = df[sorted_columns]

df.to_csv(_config.pop_write_path + '\\' + '24-72년도 인구' + '.csv', encoding=_config.euc_kr, index=False)


In [13]:
# 24 ~ 72년도 전국 인구 데이터의 연령별 인구 증감률을 구하기

import _util
import _codes
import _config
import pandas as pd
import math

# 23년도 인구 데이터 -> 전국 데이터만 뽑아오기 위한 전처리
df_23 = pd.read_csv(_config.pop_write_path + '\\' + '23년도 인구' + '.csv', encoding=_config.euc_kr)
_list = [row for idx, row in df.iterrows() if str(row['sido_cd']).count('`00')]

df = pd.DataFrame(_list)
_list = None

# 두 데이터 병합
df_list = [
    df
    , pd.read_csv(_config.pop_write_path + '\\' + '24-72년도 인구' + '.csv', encoding=_config.euc_kr)
]

df = pd.concat(df_list, ignore_index=True)

# 2024년 부터의 인구 증감률 데이터 만들기
# 연령 컬럼 뽑아내기 위한 과정
age_num = [_util.extract_number(col) for col in df.columns if not math.isinf(_util.extract_number(col))]

ages = []
for age in age_num:
    if age == 100:
        ages.append(str(age) + '세 이상')
        continue

    ages.append(str(age) + '세')

zero_age = [] # zero 나이 확인용
df_list = [] # df_list 초기화 // 24 ~ 72년까지의 sgg의 인구 표기를 위한 df
for idx, row in df.iterrows():
    if idx == 0: continue
    bf_row = df.loc[idx - 1] # 이전 데이터
    ct_row = df.loc[idx]     # 현재 데이터

    # 연령별로 나이 증감률 나누기
    change_rate = []
    for age in ages:
        nume = float(bf_row[age]) # 분자
        deno = float(ct_row[age]) # 분모
        if deno != 0: val = nume / deno # 각 나이별 증감률
        else: val = 0

        if val == 0: zero_age.append(age) 
        change_rate.append(val)

    # 0세 ~ 100세 이상 배열

    # sido_cd != '`00', 

    # 각각의 시군구 = df_23의 각각의 row,
    for rate in change_rate:
        _list = []
        for idx, row in df_23.iterrows(): # 해당 시군구에서 인구 = col, 
            sido_cd = row['sido_cd']
            sgg_cd = row['sgg_cd']
            sgg_nm = row['sgg_nm']
            gender = row['gender']

            if sido_cd == '`00': continue
            _list.append({
                'sido_cd': sido_cd
                , 'sgg_cd': sgg_cd
                , 'sgg_cd': sgg_cd
                , 'sgg_cd': sgg_cd
                , 'sgg_cd': sgg_cd
            })

            for age in ages:
                _list.append({
                    age: row[age] * rate
                })

        df = pd.DataFrame(_list)
        df_list.append(df)
    
df = pd.concat(df_list, ignore_index=False)
df.to_csv(_config.pop_write_path + '\\' + '24-72년도 시군구 인구 예측' + '.csv', index=False, encoding=_config.euc_kr)

    # 해당 연령 * change_rate *
            

        


# for num in zero_age:
#     if _util.extract_number(num) < 90:
#         print(num)
#     else: print('90 초과',num)

In [61]:
# 현재 인구 데이터 가공 파일 + 추계 인구 데이터 가공 파일 => 병합 파일 생성 

import _config
import _util
import _codes
import pandas as pd

files = _util.get_files(_config.pop_write_path)
print(files)
_list = []
for f in files:
    df = pd.read_csv(_config.pop_write_path + '\\' + f, encoding=_config.euc_kr)
    _list.append(df) 

df = pd.concat(_list, ignore_index=True)
df.to_csv(_config.pop_write_path + '\\' + 'test2' + '.csv', encoding=_config.euc_kr, index=False)

['00년도 인구.csv', '01년도 인구.csv', '02년도 인구.csv', '03년도 인구.csv', '04년도 인구.csv', '05년도 인구.csv', '06년도 인구.csv', '07년도 인구.csv', '08년도 인구.csv', '09년도 인구.csv', '10년도 인구.csv', '11년도 인구.csv', '12년도 인구.csv', '13년도 인구.csv', '14년도 인구.csv', '15년도 인구.csv', '16년도 인구.csv', '17년도 인구.csv', '18년도 인구.csv', '19년도 인구.csv', '20년도 인구.csv', '21년도 인구.csv', '22년도 인구.csv', '23년도 인구.csv', '24-72년도 인구.csv']


In [2]:
# 병합 파일 => db에 insert하기

import _config
import _util
import _codes
import pandas as pd

df = pd.read_csv(_config.pop_write_path + '\\' + 'test2' + '.csv', encoding=_config.euc_kr)

_cols = []
for col in df.columns:
    if col.count('sido_cd') > 0: val = f'\"{col}\" varchar(2)'
    elif col.count('sgg_cd') > 0: val = f'\"{col}\" varchar(5)'
    elif col.count('sgg_nm') > 0: val = f'\"{col}\" varchar(10)'
    elif col.count('gender') > 0: val = f'\"{col}\" varchar(3)'
    elif col.count('year') > 0: val = f'\"{col}\" varchar(4)'
    else: val = f'\"{col}\" float8'

    _cols.append(val)

sql = f'''
    do $$
    begin
        if exists (select 1 from pg_tables where tablename ='pop') then
        drop table pop cascade;
        end if;
        if not exists (select 1 from pg_tables where tablename ='pop') then
        create table pop(
            {', '.join(_cols)}
        );
        end if;
    end $$;
'''

_util.execute_sql(sql)

def chunker(seq, size):
    return (seq[pos:pos + size] for pos in range(0, len(seq), size))

columns = df.columns.tolist()
for idx in range(len(columns)):
    columns[idx] = f'"{columns[idx]}"'

# 데이터프레임의 모든 행을 리스트로 변환
data = df.values.tolist()

# 데이터 배치로 나누기
for chunk in chunker(data, 1000):
    values = []
    for row in chunk:
        for idx in range(len(columns)):
            if str(row[idx]).count('`') > 0:
                val = str(row[idx]).replace('`', '')
                row[idx] = f'\'{val}\''
            else:
                row[idx] = f'\'{str(row[idx])}\''
            if row[idx] == 'None':
                row[idx] = '\'\''

        values.append(f"({', '.join(row)})")
    
    sql = f'''
        do $$
        begin
            if exists (select 1 from pg_tables where tablename = 'pop') then
                insert into public.pop(
                    {','.join(columns)}
                ) values {', '.join(values)};
            end if;
        end $$;
    '''
    
    _util.execute_sql(sql)
    # try:
    #     _util.execute_sql(sql)
    # except Exception:
    #     print('sql 실행도중 에러 발생')

print("데이터 삽입 완료")



    do $$
    begin
        if exists (select 1 from pg_tables where tablename ='pop') then
        drop table pop cascade;
        end if;
        if not exists (select 1 from pg_tables where tablename ='pop') then
        create table pop(
            "sido_cd" varchar(2), "sgg_cd" varchar(5), "sgg_nm" varchar(10), "gender" varchar(3), "year" varchar(4), "0세" float8, "1세" float8, "2세" float8, "3세" float8, "4세" float8, "5세" float8, "6세" float8, "7세" float8, "8세" float8, "9세" float8, "10세" float8, "11세" float8, "12세" float8, "13세" float8, "14세" float8, "15세" float8, "16세" float8, "17세" float8, "18세" float8, "19세" float8, "20세" float8, "21세" float8, "22세" float8, "23세" float8, "24세" float8, "25세" float8, "26세" float8, "27세" float8, "28세" float8, "29세" float8, "30세" float8, "31세" float8, "32세" float8, "33세" float8, "34세" float8, "35세" float8, "36세" float8, "37세" float8, "38세" float8, "39세" float8, "40세" float8, "41세" float8, "42세" float8, "43세" float8, "44세" float8, "45세" float8, "46세" 

In [12]:
# 특정년도 시군구 데이터와 인구 데이터 병합해서 통계내기

import _util
import pandas as pd
import _config

df = pd.read_csv(_config.pop_write_path + '\\' + 'test2' + '.csv', encoding=_config.euc_kr)

cols = ['row_number() over() as fid']
for col in df.columns:
    if col.count('sgg_cd') > 0: val = f's."{col}"'
    elif col.count('sgg_nm') > 0: val = f's."{col}"'
    else: val = f'p."{col}"'
    cols.append(val)

table_nm = 'v_simple_pop_2023'
sql = f'''
    do $$
    begin
        if exists (select 1 from pg_matviews where matviewname = '{table_nm}') then
            drop materialized view {table_nm};
        end if;
        if exists (select 1 from pg_tables where tablename = 'pop') then
        create materialized view {table_nm} as
        with p_2023 as (
            select *
            from pop
            where "year" = '2023'
        ) 
        select {', '.join(cols)}, s.geom
        from simple_sgg_2023 s
        left join p_2023 p on p.sgg_cd = s.sgg_cd
        ;
        end if;
    end $$;
'''

_util.execute_sql(sql)



    do $$
    begin
        if exists (select 1 from pg_matviews where matviewname = 'v_simple_pop_2023') then
            drop materialized view v_simple_pop_2023;
        end if;
        if exists (select 1 from pg_tables where tablename = 'pop') then
        create materialized view v_simple_pop_2023 as
        with p_2023 as (
            select *
            from pop
            where "year" = '2023'
        ) 
        select row_number() over() as fid, p."sido_cd", s."sgg_cd", s."sgg_nm", p."gender", p."year", p."0세", p."1세", p."2세", p."3세", p."4세", p."5세", p."6세", p."7세", p."8세", p."9세", p."10세", p."11세", p."12세", p."13세", p."14세", p."15세", p."16세", p."17세", p."18세", p."19세", p."20세", p."21세", p."22세", p."23세", p."24세", p."25세", p."26세", p."27세", p."28세", p."29세", p."30세", p."31세", p."32세", p."33세", p."34세", p."35세", p."36세", p."37세", p."38세", p."39세", p."40세", p."41세", p."42세", p."43세", p."44세", p."45세", p."46세", p."47세", p."48세", p."49세", p."50세", p."51세", p."52세", p."53세", p

In [26]:
# 특정년도 시군구 매핑 데이터 파일로 만들기

import _util
import pandas as pd
import _config

year = 2023
file_nm = f'pop_{year}'

sql = f'''
    SELECT column_name as col_nm
    FROM information_schema.columns
    WHERE table_schema = 'public' AND table_name = 'pop';
'''

result = _util.execute_sql(sql)
if result is not None:
    col_nms = [row[0] for row in result]
    _cols = list(col_nms)
    cols = [f's.\"{col}\"' for col in _cols]
    
    sql = f'''
        select {', '.join(cols)}
        from v_simple_pop_{year} s
    '''

    result = _util.execute_sql(sql)

    df = pd.DataFrame(result)

    # df_cols = [col for col in df.columns if col != 'Unnamed: 0']
    cols = {}
    for idx in range(len(_cols)): cols[idx] = _cols[idx]
    df = pd.DataFrame(list(result))
    df.rename(columns=cols, inplace=True)

    df.to_csv(_config.pop_write_path + '\\' + file_nm + '.csv', encoding=_config.euc_kr, index=False)




    SELECT column_name as col_nm
    FROM information_schema.columns
    WHERE table_schema = 'public' AND table_name = 'pop';

[('sido_cd',), ('sgg_cd',), ('sgg_nm',), ('gender',), ('year',), ('0세',), ('1세',), ('2세',), ('3세',), ('4세',), ('5세',), ('6세',), ('7세',), ('8세',), ('9세',), ('10세',), ('11세',), ('12세',), ('13세',), ('14세',), ('15세',), ('16세',), ('17세',), ('18세',), ('19세',), ('20세',), ('21세',), ('22세',), ('23세',), ('24세',), ('25세',), ('26세',), ('27세',), ('28세',), ('29세',), ('30세',), ('31세',), ('32세',), ('33세',), ('34세',), ('35세',), ('36세',), ('37세',), ('38세',), ('39세',), ('40세',), ('41세',), ('42세',), ('43세',), ('44세',), ('45세',), ('46세',), ('47세',), ('48세',), ('49세',), ('50세',), ('51세',), ('52세',), ('53세',), ('54세',), ('55세',), ('56세',), ('57세',), ('58세',), ('59세',), ('60세',), ('61세',), ('62세',), ('63세',), ('64세',), ('65세',), ('66세',), ('67세',), ('68세',), ('69세',), ('70세',), ('71세',), ('72세',), ('73세',), ('74세',), ('75세',), ('76세',), ('77세',), ('78세',), ('79세',), ('80세',), ('81세'